# Extraction of Spindles from Neocortex recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [1]:
cd "C:/Users/AudreyHay/Documents/script"

[WinError 3] The system cannot find the path specified: 'C:/Users/AudreyHay/Documents/script'
c:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\python


In [2]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
import fnmatch

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

%matplotlib widget


# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

In [3]:
dpath = "C:/Users/AudreyHay/Documents/"
#dpath = "//10.69.168.1/crnldata/waking/audrey_hay/"
try:
    %store -r dpath
except:
    print("data not in store")
    dpath = "C:/Users/AudreyHay/Documents/"    
    #dpath ="//10.69.168.1/crnldata/waking/audrey_hay/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\CamilleData\enregistrements\25.04\sommeil après', f…

## To change  
#### (only used if sommeil Bonsaï file)

In [4]:
mice='Pink'    # mice='Pink' OR  mice='Blue'

## Load LFPs file

In [6]:
folderpath = Path(dpath)

# Load channels map 
Channels = 'SoleneMiceLFPChannels.xlsx' 
#Channels = '//10.69.168.1/crnldata/waking/audrey_hay/CamilleData/SoleneMiceLFPChannels.xlsx' 
allchannels = pd.read_excel(Channels)
Map=np.array(allchannels['MICE'])

ctx=2 # ctx=2 for PFC, ctx=4 for S1 
emg=0 

############
## Bonsai ##
############

if find_files_with_string(folderpath,  ".bin"): 
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype=np.uint16)
    All = All - int(65535/2)
    All = All.astype(np.int16)

    ### Sommeil ###
    if 'sommeil' in dpath:
        numchannel=64
        All = All.reshape(-1,numchannel) 
        Ch=32 if mice == 'Pink' else 0

        EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch 
        EMG  =  All[:, EMGch]
        
        try:
            CTXch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch 
            CTXch2=int(allchannels[mice][ctx].split(',')[1]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch
            CTX  = All[:, CTXch1]-All[:, CTXch2]
        except:
            CTXch1=int(allchannels[mice][ctx]) if mice == 'Blue' else int(allchannels[mice][2].split(',')[0])+Ch
            print('/!\ One channel only')
            CTX  =  All[:, CTXch1]      
        print(f'Bonsaï file loaded: {mice} mouse / sommeil file')
    
    ### Cheeseboard ###
    elif 'cheeseboard' in dpath:
        numchannel=32
        All = All.reshape(-1,numchannel)             
        Ch=32 if 'pink' in dpath  else 0
        mice='Pink' if 'pink' in dpath  else 'Blue'

        EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch
        EMG  =  All[:, EMGch]
        try:
            CTXch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch    
            CTXch2=int(allchannels[mice][ctx].split(',')[1]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch
            CTX  = All[:, CTXch1]-All[:, CTXch2]
        except:
            CTXch1=int(allchannels[mice][ctx]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch
            print('/!\ One CTX channel only')
            CTX  =  All[:, CTXch1]  
        print(f'Bonsaï file loaded: {mice} mouse / cheeseboard file')
        
###############
## OpenEphys ##
###############

elif find_files_with_string(folderpath,  "continuous.dat"): 
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype="int16")
    numchannel=64
    All = All.reshape(-1,numchannel)
    All = np.memmap(matching_file, mode='r', dtype='int16')
    print('Open Ephy file loaded: continuous.dat')
    Ch=32 if 'pink' in dpath  else 0
    mice='Pink' if 'pink' in dpath  else 'Blue'  

    EMGch=int(allchannels[mice][emg]) if mice == 'Blue' else int(allchannels[mice][emg])+Ch #EMG= 1st line in the excel file
    CTXch1=int(allchannels[mice][ctx].split(',')[0]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[0])+Ch     #CTX == 4th line in the excel file
    CTXch2=int(allchannels[mice][ctx].split(',')[1]) if mice == 'Blue' else int(allchannels[mice][ctx].split(',')[1])+Ch
    
    EMG  =  All[:, EMGch]
    CTX  =  All[:, CTXch1] if mice == 'Blue' else All[:, CTXch1]-All[:, CTXch2]

## OpenEphys Gaelle's Data ##
elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): 
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    All = All.reshape(-1,32)
    print('Open Ephy file loaded: RawDataChannelExtractedDS.npy')


\\10.69.168.1\crnldata\waking\audrey_hay\CamilleData\enregistrements\25.04\sommeil après
['OE_32ch_data_2024-04-25T12_24_56.bin', 'OE_32ch_timestamps_2024-04-25T12_24_56.csv', 'Spindlesproperties_CTX4Initial_AB.csv', 'Spindlesproperties_CTX4Initial_AB.pkl', 'Spindlesproperties_CTX4_AB.csv', 'Spindlesproperties_CTX4_AB.pkl', 'Spindlesproperties_PFCInitial_AB.csv', 'Spindlesproperties_PFCInitial_AB.pkl', 'Spindlesproperties_PFC_AB.csv', 'Spindlesproperties_PFC_AB.pkl', 'SWRpropertiesInitial_AB.csv', 'SWRpropertiesInitial_AB.pkl', 'SWRproperties_AB.csv', 'SWRproperties_AB.pkl', 'SWRproperties_Blue_CA1Initial_AB.xlsx', 'SWRproperties_Blue_CA1_AB.xlsx']
\\10.69.168.1\crnldata\waking\audrey_hay\CamilleData\enregistrements\25.04\sommeil après
['OE_32ch_data_2024-04-25T12_24_56.bin', 'OE_32ch_timestamps_2024-04-25T12_24_56.csv', 'Spindlesproperties_CTX4Initial_AB.csv', 'Spindlesproperties_CTX4Initial_AB.pkl', 'Spindlesproperties_CTX4_AB.csv', 'Spindlesproperties_CTX4_AB.pkl', 'Spindlesproperti

# Remove Wake signal

In [7]:
# Filter parameter :
f_lowcut = 200.
f_hicut = 400.
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction


# Filter creation :
b, a = signal.butter(N, Wn, 'band')
filt_EMG = signal.filtfilt(b, a, EMG)

# Plot
times = np.arange(0, EMG.size/fs, 1./fs)
timesmin = np.arange(0, EMG.size/fs/60, 1./fs/60)

In [8]:
# Parameter and computation of CWT
w = 4.
freq = np.linspace(200, 400, 50)
widths = w*fs / (2*freq*np.pi)
EMGcwt = signal.cwt(EMG, signal.morlet2, widths, w=w)

# Projection calculation
absEMGcwt = np.absolute(EMGcwt)
proj_EMGcwt = np.sum(absEMGcwt, axis = 0)/50
mproj_EMGcwt = np.mean(proj_EMGcwt)
sdproj_EMGcwt = np.std(proj_EMGcwt)
sd3proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt*3
sd05proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt*0.5
sd1proj_EMGcwt = mproj_EMGcwt + sdproj_EMGcwt


In [9]:
# Assigning values wake (1, 2) and sleep (0)
numpnts = EMG.size
EMGstatusRaw = np.zeros(numpnts)
for ind in range(numpnts):
    if proj_EMGcwt[ind]<sd05proj_EMGcwt:
        EMGstatusRaw[ind] = 0
    elif proj_EMGcwt[ind]>sd3proj_EMGcwt:
        EMGstatusRaw[ind] = 2
    else:
        EMGstatusRaw[ind] = 1

# Expanding borders for wake (1, 2) and sleep (0) to ±1 s around detected muscular activity
EMGstatusRaw2 = np.zeros(numpnts)
for ind in range(numpnts):
    if EMGstatusRaw[ind]>1:
       EMGstatusRaw2[ind-1000:ind+1000] = 2
    elif EMGstatusRaw[ind]==1:
        for ind2 in range(ind-1000, ind+1000):
            if ind2==numpnts:
                break
            elif EMGstatusRaw2[ind2]<2:
                EMGstatusRaw2[ind2] = 1

In [10]:
EMGStatusBoolLib = (EMGstatusRaw2>1)
EMGStatusBoolCons = (EMGstatusRaw2>0)

LFP = All[:,:]

LFPwake0 = LFP.copy()
LFPwake0[EMGStatusBoolLib,:] = 0

LFPwakeremoved = LFP.copy()
LFPwakeremoved = LFPwakeremoved[~EMGStatusBoolLib, :]

In [11]:
CTXwakeremoved = LFPwakeremoved[:,CTXch1]-LFPwakeremoved[:,CTXch2] 

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [12]:
# Filter parameter :
f_lowcut = 10.
f_hicut = 16.
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filter creation :
b, a = signal.butter(N, Wn, 'band')

filt_CTX = signal.filtfilt(b, a, CTX)
filt_CTXwakeremoved = signal.filtfilt(b, a, CTXwakeremoved)

Second on the signal for which wake times have been zeroed

In [21]:
# Parameter and computation of CWT
w = 10.
freq = np.linspace(10, 16, 6) #18)
widths = w*fs / (2*freq*np.pi)
CTXNWcwt = signal.cwt(filt_CTXwakeremoved, signal.morlet2, widths, w=w)

# Projection calculation CTX
absCTXNWcwt = np.absolute(CTXNWcwt)
proj_CTXNWcwt = np.sum(absCTXNWcwt, axis = 0)/24
sdproj_CTXcwt = np.std(proj_CTXNWcwt)
sd6proj_CTXcwt = sdproj_CTXcwt*6
#sd7proj_CTXcwt = sdproj_CTXcwt*7

In [22]:
# Conservative boolean filtering of CTX filtered signal
BooleanCons = EMGStatusBoolCons
fCTXwake0C = filt_CTX.copy()
fCTXwake0C[BooleanCons] = 0
CTXwake0C = CTX.copy()
CTXwake0C[BooleanCons] = 0
# Liberal boolean filtering of CTX filtered signal
BooleanLib = EMGStatusBoolLib
fCTXwake0L = filt_CTX.copy()
fCTXwake0L[BooleanLib] = 0
CTXwake0L = CTX.copy()
CTXwake0L[BooleanLib] = 0

# Computation of CWT
CTXcwtWake0cons = signal.cwt(fCTXwake0C, signal.morlet2, widths, w=w)
CTXcwtWake0lib = signal.cwt(fCTXwake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCTXW0Ccwt = np.absolute(CTXcwtWake0cons)
proj_CTXW0Ccwt = np.sum(absCTXW0Ccwt, axis = 0)/24
absCTXW0Lcwt = np.absolute(CTXcwtWake0lib)
proj_CTXW0Lcwt = np.sum(absCTXW0Lcwt, axis = 0)/24

## Extracting Spindles and determining main properties 

First extraction of spindle peaks, initiation, end and width

In [23]:
# 7 sd threshold
peaks, properties = find_peaks(proj_CTXW0Lcwt, width=200, height=sd6proj_CTXcwt)
properties["prominences"], properties["widths"]

# Spindles boundaries taken at 70% from peak of intensity. This means that the spindles with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CTXW0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Spindle_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

Second extraction of main frequency and power 

In [24]:
projMaxP_cwtmg = np.max(CTXcwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CTXcwtWake0lib, axis = 0)+ 10 #/2 + 8
projMaxP_cwtmg.shape

nb_Spindles = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Spindles:
    Spindle_start = int(Spindle_prop[3,tt])
    Spindle_stop = int(Spindle_prop[4,tt])
    Spindle_MaxP = projMaxP_cwtmg[Spindle_start:Spindle_stop]
    Spindle_MaxF = projMaxF_cwtmg[Spindle_start:Spindle_stop]
    data[tt, 0] = max(Spindle_MaxF).round()
    data[tt, 1] = max(Spindle_MaxP).round()
    data[tt, 2] = round(sum(Spindle_MaxF)/len(Spindle_MaxF))
    data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))

param_Spindle = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSpindle_prop = Spindle_prop.transpose()
pd_prop_Spindle = pd.DataFrame(tSpindle_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Spindle = pd.concat([pd_prop_Spindle, param_Spindle], axis=1)

Spindle_peak = peaks
Spindle_start = Spindle_prop[3,:].astype(int)
Spindle_end = Spindle_prop[4,:].astype(int)

C:\Users\Manip2\AppData\Local\Temp\ipykernel_11800\2656106730.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Spindle_MaxP).round()
C:\Users\Manip2\AppData\Local\Temp\ipykernel_11800\2656106730.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))
C:\Users\Manip2\AppData\Local\Temp\ipykernel_11800\2656106730.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Spindle_MaxP)/len(Spindle_MaxP))


### Store the results in All_Spindle_prop pd dataframe and save as excel for post processing.

In [25]:
filename = folderpath / f'Spindlesproperties_{mice}_{Map[ctx]}Initial_AB.xlsx'
All_Spindle.to_excel(filename)

filename = folderpath / f'Spindlesproperties_{mice}_{Map[ctx]}_AB.xlsx'
All_Spindle.to_excel(filename)

### Display

ephys viewer to check Spindle detection

In [26]:
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Spindle_peak, 1: Spindle_start, 2: Spindle_end}
#and asign them to some channels each
scatter_channels = {0: [1, 2], 1: [0, 1], 2: [0, 1]}

combined = np.stack([CTX, filt_CTX*5, proj_CTXW0Ccwt*5, proj_CTXW0Lcwt*5], axis = 1)
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel

view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch2', 'color'] = '#ff5500'
view1.by_channel_params['ch3', 'color'] = '#ffffff'

#view1.by_channel_params['ch0', 'offset'] = 0
#view1.by_channel_params['ch1', 'offset'] = -1
#view1.by_channel_params['ch2', 'offset'] = -1
#view1.by_channel_params['ch3', 'offset'] = -1

#view1.by_channel_params['ch0', 'gain'] = .001
#view1.by_channel_params['ch1', 'gain'] = .1
#view1.by_channel_params['ch2', 'gain'] = .1
#view1.by_channel_params['ch3', 'gain'] = .1

#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

debug True
QT_MODE PyQt5
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0.0 s
refresh duration for  0

0